In [1]:
import pandas as pd
import json

file_path = 'terrorism_dataset.csv'
output_json_path = 'waffle_data.json'
K = 4

df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
df = df[['attacktype1_txt', 'weaptype1_txt']].copy()
df = df.dropna()
df = df[df['weaptype1_txt'] != 'Unknown']
df = df[df['attacktype1_txt'] != 'Unknown']

top_5_attacks = df['attacktype1_txt'].value_counts().head(5).index.tolist()

json_output = []

for attack_type in top_5_attacks:
    subset = df[df['attacktype1_txt'] == attack_type]
    weapon_counts = subset['weaptype1_txt'].value_counts()
    top_weapons = weapon_counts.head(K)
    other_count = weapon_counts.iloc[K:].sum()
    
    weapons_data = []
    for weapon, count in top_weapons.items():
        weapons_data.append({"label": weapon, "value": int(count)})
    if other_count > 0:
        weapons_data.append({"label": "Others", "value": int(other_count)})

    json_output.append({
        "category": attack_type,
        "total_events": int(subset.shape[0]),
        "data": weapons_data
    })

with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, indent=4)